In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cp drive/MyDrive/REVIDE_inside.zip /content

In [4]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from glob import glob
import random
from PIL import Image
import random
import datetime, zipfile
from tqdm import tqdm
import shutil

import tensorflow as tf
from keras.layers import * 
from keras import Input
from keras.losses import mean_squared_error
from keras.initializers import glorot_normal,random_normal
from keras.regularizers import L2
from keras.metrics import MeanSquaredError
from keras.activations import relu
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LambdaCallback
from keras import backend as K

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
# from tensorflow_addons.layers import InstanceNormalizationn
from tensorflow.image import psnr,ssim

from sklearn.model_selection import train_test_split

In [5]:
!unzip REVIDE_inside.zip

Archive:  REVIDE_inside.zip
   creating: Test/
   creating: Test/gt/
   creating: Test/gt/C005/
 extracting: Test/gt/C005/00000.JPG  
 extracting: Test/gt/C005/00001.JPG  
 extracting: Test/gt/C005/00002.JPG  
 extracting: Test/gt/C005/00003.JPG  
 extracting: Test/gt/C005/00004.JPG  
 extracting: Test/gt/C005/00005.JPG  
 extracting: Test/gt/C005/00006.JPG  
 extracting: Test/gt/C005/00007.JPG  
 extracting: Test/gt/C005/00008.JPG  
 extracting: Test/gt/C005/00009.JPG  
 extracting: Test/gt/C005/00010.JPG  
 extracting: Test/gt/C005/00011.JPG  
 extracting: Test/gt/C005/00012.JPG  
 extracting: Test/gt/C005/00013.JPG  
 extracting: Test/gt/C005/00014.JPG  
 extracting: Test/gt/C005/00015.JPG  
 extracting: Test/gt/C005/00016.JPG  
 extracting: Test/gt/C005/00017.JPG  
 extracting: Test/gt/C005/00018.JPG  
 extracting: Test/gt/C005/00019.JPG  
 extracting: Test/gt/C005/00020.JPG  
 extracting: Test/gt/C005/00021.JPG  
 extracting: Test/gt/C005/00022.JPG  
 extracting: Test/gt/C005/0002

In [6]:
if not os.path.exists('Revide_inside'):
    os.makedirs('Revide_inside')
    if not os.path.exists('Revide_inside/Train'):
        os.makedirs('Revide_inside/Train')
    if not os.path.exists('Revide_inside/Test'):
        os.makedirs('Revide_inside/Test')
%mkdir Revide_inside/Train/hazy Revide_inside/Train/gt 
%mkdir Revide_inside/Test/hazy Revide_inside/Test/gt         

In [7]:
hazy_path=os.path.join('Train','hazy')
hazy_images=os.listdir(hazy_path)
for i in tqdm(hazy_images):
    for j in os.listdir(os.path.join(hazy_path,i)):
        shutil.copy(f'{os.path.join(hazy_path,i)}/{j}','Revide_inside/Train/hazy')

100%|██████████| 42/42 [00:00<00:00, 66.09it/s]


In [8]:
hazy_path=os.path.join('Test','hazy')
hazy_images=os.listdir(hazy_path)
for i in tqdm(hazy_images):
    for j in os.listdir(os.path.join(hazy_path,i)):
        shutil.copy(f'{os.path.join(hazy_path,i)}/{j}','Revide_inside/Test/hazy')

100%|██████████| 6/6 [00:00<00:00, 62.77it/s]


In [9]:
gt_path=os.path.join('Train','gt')
gt_images=os.listdir(gt_path)
for i in tqdm(gt_images):
    for j in os.listdir(os.path.join(gt_path,i)):
        shutil.copy(f'{os.path.join(gt_path,i)}/{j}','Revide_inside/Train/gt')
        
gt_test_path=os.path.join('Test','gt')
gt_test_images=os.listdir(gt_test_path)
for i in tqdm(gt_test_images):
    for j in os.listdir(os.path.join(gt_test_path,i)):
        shutil.copy(f'{os.path.join(gt_test_path,i)}/{j}','Revide_inside/Test/gt')        

100%|██████████| 6/6 [00:00<00:00, 24.08it/s]


In [12]:
class VideoDehazing:
    def __init__(self, train_hazed_path, train_gt_path, test_hazed_path, test_gt_path):
        self.train_hazed_path=train_hazed_path
        self.train_gt_path=train_gt_path
        self.test_hazed_path=test_hazed_path
        self.test_gt_path=test_gt_path
        
    def preprocess(self):
        x_train=np.array([tf.image.resize(
            tf.io.decode_jpeg(
                tf.io.read_file(
                os.path.join(self.train_hazed_path,f)),channels=3),
            (384,384))for f in os.listdir(self.train_hazed_path)])
        y_train=np.array([tf.image.resize(
            tf.io.decode_jpeg(
                tf.io.read_file(
                os.path.join(self.train_gt_path,f)),channels=3),
            (384,384))for f in os.listdir(self.train_gt_path)])
        x_test=np.array([tf.image.resize(
            tf.io.decode_jpeg(
                tf.io.read_file(
                os.path.join(self.test_hazed_path,f)),channels=3),
            (384,384))for f in os.listdir(self.test_hazed_path)])
        y_test=np.array([tf.image.resize(
            tf.io.decode_jpeg(
                tf.io.read_file(
                os.path.join(self.test_gt_path,f)),channels=3),
            (384,384))for f in os.listdir(self.test_gt_path)])
        
        x_train, x_val, y_train, y_val=train_test_split(x_train,y_train,test_size=0.1, random_state=42)
        
        x_train=x_train/255.0
        y_train=y_train/255.0
        x_val=x_val/255.0
        y_val=y_val/255.0
        x_test=x_test/255.0
        y_test=y_test/255.0
        
        self.data_gen=ImageDataGenerator()
        self.train_gen=self.data_gen.flow(x_train,y_train,batch_size=32)
        self.val_gen = self.data_gen.flow(x_val,y_val,batch_size=32)
        self.test_gen=self.data_gen.flow(x_test,y_test,batch_size=32)
    
    def custom_loss(self,y_true,y_pred):
        mse=K.mean(K.square(y_true-y_pred))
        psnr=-10*K.log(mse)/K.log(10.0)
        ssim_=ssim(y_true,y_pred,2.0)
        return mse + (1-ssim_)+(1-psnr)
    
    
    def create_model(self):
        
        k_init=random_normal(mean=0.0,stddev=0.01, seed=101)
        regularizer=L2(1e-4)
        b_init=tf.constant_initializer()
        inputs = Input(shape = [None, 384, 384, 3])     
        # Autoencoder Başlangıç Encoder
        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3,
                strides = 1,
                padding = 'same',
                kernel_initializer = k_init,
                activation = 'relu',
                bias_initializer = b_init,
                kernel_regularizer = regularizer))(inputs)

        conv=LayerNormalization()(conv)

        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3,
                strides = 1,
                padding = 'same',
                kernel_initializer = k_init,
                activation = 'relu',
                bias_initializer = b_init,
                kernel_regularizer = regularizer))(conv)

        conv=LayerNormalization()(conv)            

        conv_up = TimeDistributed(Conv2D(filters = 128,
                    kernel_size = 3,
                    strides = 2,
                    padding = 'same',
                    kernel_initializer = k_init,
                    activation = 'relu',
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer))(conv)

        conv_up=LayerNormalization()(conv_up)

        conv_up = TimeDistributed(Conv2D(filters = 128,
                    kernel_size = 3,
                    strides = 2,
                    padding = 'same',
                    kernel_initializer = k_init,
                    activation = 'relu',
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer))(conv_up)

        conv_up=LayerNormalization()(conv_up)
        # Residual Bloklar                                
        conv1_1 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1,
                    padding = 'same',
                    kernel_initializer = k_init,
                    return_sequences=True,
                bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv_up)
        conv1_1=LayerNormalization()(conv1_1)

        conv1_2 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1,
                    padding = 'same',
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv1_1)
        conv1_2=LayerNormalization()(conv1_2)

        conv1_3 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1, 
                    padding = 'same', 
                    return_sequences=True,
                    kernel_initializer = glorot_normal(seed = 101),
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv1_2)

        conv1_3=LayerNormalization()(conv1_3)
        conc1 = tf.add(conv1_3, conv1_1)
        conv1 = relu(conc1)

        conv2_1 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1,
                    padding = 'same',
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv1)
        conv2_1=LayerNormalization()(conv2_1)

        conv2_2 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1,
                    padding = 'same',
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv2_1)
        conv2_2=LayerNormalization()(conv2_2)

        conv2_3 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1,
                    padding = 'same',
                    return_sequences=True,
                    kernel_initializer = glorot_normal(seed = 101),
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv2_2)
        conv2_3=LayerNormalization()(conv2_3)
        conc2 = tf.add(conv2_3, conv2_1)
        conv2 = relu(conc2)

        conv3_1 = ConvLSTM2D(filters = 64,
                    kernel_size = 3,
                    strides = 1,
                    padding = 'same',
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv2)
        conv3_1=LayerNormalization()(conv3_1)

        conv3_2 = ConvLSTM2D(filters = 64,
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv3_1)

        conv3_2=LayerNormalization()(conv3_2)

        conv3_3 = ConvLSTM2D(filters = 64,
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv3_2)

        conv3_3=LayerNormalization()(conv3_3)

        conv3_4 = ConvLSTM2D(filters = 64,
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv3_3)

        conv3_4=LayerNormalization()(conv3_4)

        conv3_5 = ConvLSTM2D(filters = 64, 
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    return_sequences=True,
                    kernel_initializer = glorot_normal(seed = 101),
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv3_4)

        conv3_5=LayerNormalization()(conv3_5)

        conc3 = tf.add(conv3_5, conv3_1)
        conv3 = relu(conc3)

        conv4_1 = ConvLSTM2D(filters = 64,
                    kernel_size = 3, 
                    strides = 1, padding = 'same', 
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv3)

        conv4_1=LayerNormalization()(conv4_1)

        conv4_2 = ConvLSTM2D(filters = 64, 
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv4_1)

        conv4_2=LayerNormalization()(conv4_2)

        conv4_3 = ConvLSTM2D(filters = 64,
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    kernel_initializer = k_init,
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv4_2)

        conv4_3=LayerNormalization()(conv4_3)

        conv4_4 = ConvLSTM2D(filters = 64, 
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    kernel_initializer = k_init, 
                    return_sequences=True,
                    bias_initializer = b_init,
                    kernel_regularizer = regularizer)(conv4_3)

        conv4_4=LayerNormalization()(conv4_4)

        conv4_5 = ConvLSTM2D(filters = 64,
                    kernel_size = 3, 
                    strides = 1, 
                    padding = 'same', 
                    return_sequences=True,
                    kernel_initializer =glorot_normal(seed = 101),
                    bias_initializer = b_init, 
                    kernel_regularizer = regularizer)(conv4_4)

        conv4_5=LayerNormalization()(conv4_5)

        conc4 = tf.add(conv4_5, conv4_1)
        conv4 = relu(conc4)
        #Autoencoder Decoder    
        deconv = TimeDistributed(Conv2DTranspose(filters = 64,
                            kernel_size = 3, 
                            strides = 2, 
                            padding = 'same',
                            kernel_initializer = glorot_normal(seed = 101),
                            kernel_regularizer = regularizer))(conv4)

        deconv=LayerNormalization()(deconv)

        deconv = TimeDistributed(Conv2DTranspose(filters = 64,
                            kernel_size = 3,
                            strides = 2, 
                            padding = 'same', 
                            kernel_initializer = glorot_normal(seed = 101),
                            kernel_regularizer = regularizer))(deconv)

        deconv=LayerNormalization()(deconv)                        

        conv = TimeDistributed(Conv2D(filters = 64, 
                kernel_size = 3, 
                strides = 1, 
                padding = 'same',
                kernel_initializer = k_init,
                activation = 'relu',
                bias_initializer = b_init,
                kernel_regularizer = regularizer))(deconv)

        conv=LayerNormalization()(conv)

        conv = TimeDistributed(Conv2D(filters = 3,
                kernel_size = 3,
                strides = 1,
                padding = 'same',
                kernel_initializer = glorot_normal(seed = 101),
                bias_initializer = b_init,
                kernel_regularizer = regularizer))(conv)

        conc = tf.add(conv, inputs)
        gman_output = relu(conc) #Autoencoder Sonu

        #Paralel Ağ Başlangıç    
        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3, 
                dilation_rate = 4, 
                padding = 'same',
                kernel_initializer = k_init,
                activation = 'relu',
                kernel_regularizer = regularizer))(inputs)

        conv=LayerNormalization()(conv)

        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3, 
                dilation_rate = 2,
                padding = 'same', 
                kernel_initializer = k_init,
                activation = 'relu',
                kernel_regularizer = regularizer))(conv)

        conv=LayerNormalization()(conv)            

        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3, 
                dilation_rate = 2,
                padding = 'same',
                kernel_initializer = k_init,
                activation = 'relu',
                kernel_regularizer = regularizer))(conv)

        conv=LayerNormalization()(conv)

        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3, 
                dilation_rate = 1,
                padding = 'same', 
                kernel_initializer = k_init, 
                activation = 'relu',
                kernel_regularizer = regularizer))(conv)

        conv=LayerNormalization()(conv)            

        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3, 
                dilation_rate = 1, 
                padding = 'same', 
                kernel_initializer = k_init,
                activation = 'relu',
                kernel_regularizer = regularizer))(conv)

        conv=LayerNormalization()(conv)            

        conv = TimeDistributed(Conv2D(filters = 64,
                kernel_size = 3,
                dilation_rate = 1,
                padding = 'same', 
                kernel_initializer = k_init,
                activation = 'relu',
                kernel_regularizer = regularizer))(conv)

        conv=LayerNormalization()(conv)

        deconv = TimeDistributed(Conv2DTranspose(filters = 64,
                            kernel_size = 3, 
                            dilation_rate = 4, 
                            padding = 'same', 
                            kernel_initializer = glorot_normal(seed = 101),
                            activation = 'relu', 
                            kernel_regularizer = regularizer))(conv)

        deconv=LayerNormalization()(deconv)

        conv = TimeDistributed(Conv2D(filters = 3,
                kernel_size = 3,
                strides = 1, 
                padding = 'same', 
                kernel_initializer = glorot_normal(seed = 101),
                kernel_regularizer = regularizer))(deconv)
        #Paralel Ağ Sonu
        conc = tf.add(conv, inputs)
        pn_output = relu(conc)

        output = tf.add(gman_output, pn_output)        
        self.model=Model(inputs,output)

        self.model.compile(optimizer=Adam(learning_rate=1e-4), loss=self.custom_loss)
        
    def show_results(self, epochs, logs):
        if epochs%10==0:
            hazed_img=self.x_val[0]
            gt_img=self.y_val[0]
            dehazed_img=self.model.predict(hazed_img[np.newaxis,:,:,:])[0]
            self.model.save('saved_model.h5')
            plt.figure(figsize=(20,10))
            display_list=[hazed_img, gt_img, dehazed_img]
            title=['Hazy Image', 'Ground Truth', 'Dehazed Image']

            for i in range(3):
                plt.subplot(1,3,i+1)
                plt.title(title[i])
                plt.imshow(display_list[i])
                plt.axis('off')
            plt.show()

In [13]:
train_hazed_path='Revide_inside/Train/hazy'
train_gt_path='Revide_inside/Train/gt'
test_hazed_path='Revide_inside/Test/hazy'
test_gt_path='Revide_inside/Test/gt'

dehazer=VideoDehazing(train_hazed_path,train_gt_path,test_hazed_path,test_gt_path)

dehazer.preprocess()

dehazer.create_model()

callback=LambdaCallback(on_epoch_end=dehazer.show_results)

dehazer.model.fit_generator(dehazer.train_gen, epochs=50, validation_data=dehazer.val_gen, callbacks=[callback])

<ipython-input-13-8c98d954fd55>:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  dehazer.model.fit_generator(dehazer.train_gen, epochs=50, validation_data=dehazer.val_gen, callbacks=[callback])


Epoch 1/50


ResourceExhaustedError: ignored